In [16]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Behavior and personality prediction using HCP movie-watching data

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pickle
import os
from cc_utils import _get_clip_labels
from gru.rb_utils import (
    _get_results, get_cv_tbest, cv_agg,
    get_grid_acc, get_best_param,
    get_test_tbest, get_test_predictions,
    get_static, get_static_predictions)

# plot
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = 'plotly_white'
from plot_utils import (
    _hex_to_rgb, _plot_ts,
    _add_box, _highlight_max)

colors = px.colors.qualitative.Plotly
ccc = {'Train': '#D55E00', 
    'Validation': '#CC79A7',
    'Test': '#CC79A7'}

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


## Important parameters for visualization

In [2]:
""" score: accuracy measure
    
    options:
        's': Spearman rank correlation
        'p': Pearson correlation
        'mse': mean-squared error
"""
score = 's'
pretty_score = 'Spearman rank correlation'

""" criteria for hyperparameter selection
    note: predictions are made for each clip
    
    options:
        'max': based on 'max/min' score across clips
        'mean': based on 'mean' score across clips
"""
criteria = 'mean'

## Model parameters and clip information

In [3]:
roi = 300
net = 7
subnet = 'wb'

zscore = 1
k_fold = 5
train_size = 100

In [4]:
clip_y = _get_clip_labels()
clip_y['testretest'] = 0
k_clip = len(np.unique(list(clip_y.values())))

clip_names = np.zeros(k_clip).astype(str)
clip_names[0] = 'testretest'
for key, item in clip_y.items():
    if item != 0:
        clip_names[item] = key
        
select_bhv = ['PMAT24_A_CR', 'PicVocab_Unadj',
              'NEOFAC_A', 'NEOFAC_E', 'NEOFAC_C',
              'NEOFAC_N', 'NEOFAC_O']

pretty_bhv = {}
for bhv in select_bhv:
    pretty_bhv[bhv] = bhv
pretty_bhv['PMAT24_A_CR'] = 'Fluid Intelligence'
pretty_bhv['PicVocab_Unadj'] = 'Verbal IQ'
print('Behavior / personality scores include:')
for bhv in select_bhv:
    print(pretty_bhv[bhv])

Behavior / personality scores include:
Fluid Intelligence
Verbal IQ
NEOFAC_A
NEOFAC_E
NEOFAC_C
NEOFAC_N
NEOFAC_O


In [5]:
class GRU_ARGS():
    roi = roi
    net = net
    subnet = subnet
    
    zscore = zscore
    k_fold = k_fold
    train_size = train_size
    
    # model
    model_type = 'gru'
    k_param = [16]
    param_str = 'k_hidden'
    k_layers = 3
    batch_size = 30
    num_epochs = [20]
    
    l2 = 0.0
    dropout=0.000001
    lr=0.001
    
    RES_DIR = 'results/bhv_{}'.format(model_type)
    
    select_bhv = select_bhv
    clip_names = clip_names
    clip_y = clip_y

In [6]:
models = ['gru']
pretty_model = {'gru': 'Gated Recurrent Unit'}
print('Models include:')
for model in models:
    print(f'{model.upper():5}: {pretty_model[model]}')

Models include:
GRU  : Gated Recurrent Unit


In [7]:
args = {}
args['gru'] = GRU_ARGS()
K_RUNS = 4

## Find best hyperparameters for behavior prediction

In [9]:
opt_hyperparam = {}
for model in models:
    opt_hyperparam[model] = {}
    for ii, bhv in enumerate(select_bhv):
        opt_hyperparam[model][bhv] = {}
        if model in ['gru']:
            num_epoch, k_param = get_best_param(
                args[model], bhv, score = score, criteria = criteria)
            print('{:4} - {:20}: num_epochs = {:02}, {:10} = {}'.format(
                model, pretty_bhv[bhv], num_epoch, 
                args[model].param_str, k_param))
            opt_hyperparam[model][bhv]['num_epochs'] = num_epoch
            opt_hyperparam[model][bhv][args[model].param_str] = k_param
        elif model in ['cpm', 'bbs']:
            k_param = get_best_param(
                args[model], bhv, score = score, criteria = criteria)
            print('{:4} - {:20}: {} = {}'.format(
                model, bhv, args[model].param_str, k_param))
            opt_hyperparam[model][bhv][args[model].param_str] = k_param
    print('---')

16 20
gru  - Fluid Intelligence  : num_epochs = 20, k_hidden   = 16
16 20
gru  - Verbal IQ           : num_epochs = 20, k_hidden   = 16
16 20
gru  - NEOFAC_A            : num_epochs = 20, k_hidden   = 16
16 20
gru  - NEOFAC_E            : num_epochs = 20, k_hidden   = 16
16 20
gru  - NEOFAC_C            : num_epochs = 20, k_hidden   = 16
16 20
gru  - NEOFAC_N            : num_epochs = 20, k_hidden   = 16
16 20
gru  - NEOFAC_O            : num_epochs = 20, k_hidden   = 16
---


## Get test score and prediction based on optimal hyperparameters

In [10]:
def get_score_and_predictions(args, star_hyper, score):
    """get test accuracy and predictions at tbest
        
    Args:
        bhv_measures
        star_hyper: saved optimal hyperparameters
        score: 'p', 's', or 'mse'
        
    Return:
        test_acc: best score
        y_hat: predictions
        y: true scores
    """
    bhv_measures = args.select_bhv
    k_clips = len(clip_names)
    
    test_acc = {}
    y_hat, y = {}, {}
    fig = {}
    for ii, bhv in enumerate(bhv_measures):
        
        if args.model_type in ['gru']:
            num_epochs = star_hyper[bhv]['num_epochs']
            k_param = star_hyper[bhv][args.param_str]
            r = _get_results(args, bhv, num_epochs, k_param)
            
            clip_test_acc = np.zeros(len(clip_names))
            for jj, clip in enumerate(clip_names):
                _, V = get_test_tbest(args, clip, r, score)
                clip_test_acc[jj] = V

            if score != 'mse':
                best_clip = clip_names[np.argmax(clip_test_acc)]
                test_acc[bhv] = np.max(clip_test_acc)
            else:
                best_clip = clip_names[np.argmin(clip_test_acc)]
                test_acc[bhv] = np.min(clip_test_acc)

            y_hat[bhv], y[bhv] = get_test_predictions(args, best_clip, 
                                                      r, score)
            
        elif args.model_type in ['cpm', 'bbs']:
            k_param = star_hyper[bhv][args.param_str]
            r = _get_results(args, bhv, k_param = k_param)

            c_acc = np.zeros(k_clips)
            for jj, clip in enumerate(clip_names):
                _, V = get_static(args, clip, r, score)
                c_acc[jj] = V

            if score != 'mse':
                best_clip = clip_names[np.argmax(c_acc)]
            else:
                best_clip = clip_names[np.argmin(c_acc)]

            _, test_acc[bhv] = get_static(args, best_clip, r, score,
                                          mode='test')

            y_hat[bhv], y[bhv] = get_static_predictions(args, best_clip, 
                                                        r, score)
        
        # df for px
        df = pd.DataFrame({'true': y[bhv],
                           'predicted':y_hat[bhv]})
        fig[bhv] = px.scatter(df, x='true', 
            y='predicted', trendline='ols')
        fig[bhv].update_layout(height=350, width=400,
                               title_text=pretty_bhv[bhv])
        fig[bhv].update_xaxes(dtick=0.2)
        fig[bhv].update_yaxes(dtick=0.2)
    
    return test_acc, y_hat, y, fig

In [11]:
test_acc = {}
save_y_hat, save_y = {}, {}
for model in models:
    t_acc, pred, true, _ = get_score_and_predictions(
        args[model], opt_hyperparam[model], score)
    test_acc[model] = t_acc
    save_y_hat[model] = pred
    save_y[model] = true

In [12]:
best_score = {'bhv':[]}
for model in models:
    best_score[model.upper()] = []
for ii, bhv in enumerate(select_bhv):
    best_score['bhv'].append(pretty_bhv[bhv])
    for model in models:
        best_score[model.upper()].append(f'{test_acc[model][bhv]:.4f}')
df = pd.DataFrame(best_score)
df

,bhv,GRU
0,Fluid Intelligence,0.3175
1,Verbal IQ,0.2567
2,NEOFAC_A,0.0772
3,NEOFAC_E,0.3847
4,NEOFAC_C,0.3200
5,NEOFAC_N,0.2307
6,NEOFAC_O,0.2762


In [13]:
fig = go.Figure()
for model in models:
    y = []
    x = []
    for bhv in select_bhv:
        x.append(pretty_bhv[bhv])
        y.append(test_acc[model][bhv])
    bar = go.Bar(name = model.upper(),
                 x = x, y = y)
    fig.add_trace(bar)
    
fig.update_layout(barmode = 'group')
fig.update_xaxes(title_text = 'Model')
fig.update_yaxes(title_text = pretty_score)
fig.show()

In [14]:
fig = go.Figure()
for bhv in select_bhv:
    y = []
    x = []
    for model in models:
        x.append(model.upper())
        y.append(test_acc[model][bhv])
    bar = go.Bar(name = pretty_bhv[bhv],
                 x = x, y = y)
    fig.add_trace(bar)
    
fig.update_layout(barmode = 'group')
fig.update_xaxes(title_text = 'Behavior/Personality score')
fig.update_yaxes(title_text = pretty_score)
fig.show()

In [15]:
for bhv in select_bhv:
    plot_y, plot_y_hat = [], []
    plot_tag = []
    for model in models:
        plot_y += list(save_y[model][bhv])
        plot_y_hat += list(save_y_hat[model][bhv])
        k_sub = len(save_y[model][bhv])
        plot_tag += [model]*k_sub

    # df for px
    df = pd.DataFrame(
        {'true': plot_y, 'predicted': plot_y_hat,
         'tag': plot_tag})
    fig = px.scatter(df, x='true', y='predicted', 
                     color='tag', trendline='ols')
    fig.update_layout(height=400, width=480,
                      title_text=pretty_bhv[bhv],
                      font_color='black',
                      legend_font_size=20)
    fig.update_xaxes(dtick=0.2, tickfont=dict(size=20),
                     title=dict(text='True',
                                font_size=20))
    fig.update_yaxes(dtick=0.2, tickfont=dict(size=20),
                     title=dict(text='Predicted',
                                font_size=20))
    fig.show()